#### Day 12 - B
Each plot is defined by groups of letter touching
Price of a plot is the Area (count of chars) * Sides
Get the total price


In [66]:
#Import Libraries and settings

settings = {
    "day": "12",
    "test_data": 0
}

plot_ids = {
    "plots": {},
    "history": []
}

In [67]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    grid = [[x for x in xs] for xs in lines]

    return grid

grid = load_input(settings)
GRID_WIDTH = len(grid[0])
GRID_HEIGHT = len(grid[0])

In [68]:
#Get all neighbouring spaces that match the target character and are not excluded
def get_neighbours(grid, loc, target=None, exclude=[]):

    #Get all neighbours in the grid
    nb_locs = []
    if loc[1] > 0:  
        nb_locs.append((loc[0], loc[1] - 1)) #North
    if loc[1] < (GRID_HEIGHT - 1):
        nb_locs.append((loc[0], loc[1] + 1)) #South
    if loc[0] < (GRID_WIDTH - 1):  
        nb_locs.append((loc[0] + 1, loc[1])) #East
    if loc[0] > 0:
        nb_locs.append((loc[0] - 1, loc[1])) #West

    valid_nbs = []
    for nb in nb_locs:
        #Check new loc is the target character
        if (grid[nb[1]][nb[0]] == target) or target is None:
            #Check the loc is not excluded
            if nb not in exclude:
                valid_nbs.append(nb)

    return valid_nbs

#Get all plot spaces in the grid for a given location in the plot
def get_plot_spaces(grid, loc):

    #Plot character
    plot_char = grid[loc[1]][loc[0]]
    #Starting state
    to_process = [loc]
    plot_locs = [loc]

    while to_process:
        #List for new neighbours found in this iteration
        nbs = []
        
        for space in to_process:
            #Get all unprocessed neighbours for this space
            space_neighbours = get_neighbours(grid, space, target=plot_char, exclude=plot_locs)
            #print(space, space_neighbours)
            #Update list of plot locations to avoid processing the same space multiple times
            plot_locs += space_neighbours
            #Update the neighbours list for this iteration
            nbs += space_neighbours

        #Update spaces to process for next loop iteration
        to_process = nbs

    return plot_locs

#For a given space, give the plot an ID
def id_plot(grid, loc):

    global plot_ids

    #Plot character
    plot_char = grid[loc[1]][loc[0]]

    #Get new plot_id
    id_suffix = 1
    while True:
        plot_id = plot_char + "_" + str(id_suffix)
        if plot_id not in plot_ids["plots"].keys():
            break
        id_suffix += 1

    plot_spaces = get_plot_spaces(grid, loc)

    plot_ids["history"] += plot_spaces
    plot_ids["plots"][plot_id] = plot_spaces

#Record all plots in the grid with an id
def categorise_grid(grid):

    global plot_ids

    for idx_y, line in enumerate(grid):
        for idx_x, space in enumerate(grid):
            loc = (idx_x, idx_y)
            if loc not in plot_ids["history"]:
                id_plot(grid, loc)

In [69]:
#Run the categorise code
categorise_grid(grid)

In [70]:
#Get the plot area (Number of spaces in the plot)
def get_area(plot):
    return len(plot)

#Get the perimeter (Number of neighbouring spaces not in the plot)
def get_perimeter(grid, plot):
    #Get plot character
    plot_char = grid[plot[0][1]][plot[0][0]]

    subtotal = 0

    for space in plot:
        res = (4 - len(get_neighbours(grid, space, target=plot_char)))
        subtotal += res

    return subtotal

#Find number of corners in the plot
def get_sides(grid, plot):

    #Get plot character
    plot_char = grid[plot[0][1]][plot[0][0]]

    #Get all corner ids with count
    plot_corners = {}
    for space in plot:
        x = space[0]
        y = space[1]
        seen_corners = [(x, y), (x+1, y), (x, y+1), (x+1, y+1)]
        for seen_corner in seen_corners:
            if seen_corner not in plot_corners.keys():
                plot_corners[seen_corner] = 1
            else:
                plot_corners[seen_corner] += 1

    #Count number of sides
    #Corners seen by an odd number of spaces in the plot
    sides = 0
    for c in plot_corners.keys():
        if plot_corners[c] % 2 == 1:
            sides += 1
        else:
            #Check shared corner if corner is seen by 2 spaces and surrounding spaces are in the grid bounds
            if plot_corners[c] == 2:
                if (c[0] > 0 and c[0] < GRID_WIDTH) and (c[1] > 0 and c[1] < GRID_HEIGHT):
                    if (grid[c[1]][c[0]] == plot_char and grid[c[1]-1][c[0]-1] == plot_char):
                        sides += 2
                    elif (grid[c[1]-1][c[0]] == plot_char and grid[c[1]][c[0]-1] == plot_char):
                        sides += 2

    return sides
    
#Get the price for a given plot_id
def price_fence(grid, plot_id):
    global plot_ids
    plot = plot_ids["plots"][plot_id]
    return get_area(plot) * get_sides(grid, plot)

In [71]:
#Price all the fences and sum the result
def price_grid(grid):
    total_price = 0

    for plot_id in plot_ids["plots"].keys():
        total_price += price_fence(grid, plot_id)

    return total_price

In [72]:
price_grid(grid)

893676